In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 20.9 MB/s eta 0:00:0000:01


In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

data_path = '/kaggle/input/options-data/'

In [4]:
data = pd.read_csv(data_path + 'data_cleaned.csv')
data.rename(columns={'Unnamed: 0': 'DateTime'}, inplace=True)
data.set_index('DateTime', inplace=True)
opt_data = data[["AMZN_OPT","FB_OPT","EA_OPT","NFLX_OPT","ADBE_OPT"]].dropna(how="all").join(data[["rf"]],how="inner").join(data[["AMZN","FB","EA","NFLX","ADBE"]])
strike_price_dict = {"AMZN":1757.5,"FB":197.5,"EA":105,"NFLX":305,"ADBE":325}
expire_time = "2019/12/20 16:00"
opt_expire_time = datetime.strptime(expire_time,"%Y/%m/%d %H:%M")
opt_data.head()

,AMZN_OPT,FB_OPT,EA_OPT,NFLX_OPT,ADBE_OPT,rf,AMZN,FB,EA,NFLX,ADBE
DateTime,,,,,,,,,,,
2019/10/30 9:30,NaN,NaN,0.53,6.56,NaN,1.8191,1762.0000,187.7200,93.630,286.9500,274.105
2019/10/30 9:40,NaN,NaN,0.60,6.96,NaN,1.8103,1768.6140,188.4900,94.210,288.7200,274.365
2019/10/30 9:50,NaN,NaN,0.56,NaN,NaN,1.8138,1766.4446,188.1050,93.140,289.1200,274.015
2019/10/30 10:00,NaN,NaN,0.46,7.24,NaN,1.8138,1766.4600,187.8081,92.770,287.8700,273.870
2019/10/30 10:10,NaN,NaN,0.45,NaN,NaN,1.8120,1768.1724,187.9650,93.085,288.8869,273.430


In [5]:
opt_data.tail()

,AMZN_OPT,FB_OPT,EA_OPT,NFLX_OPT,ADBE_OPT,rf,AMZN,FB,EA,NFLX,ADBE
DateTime,,,,,,,,,,,
2019/12/16 10:50,14.60,2.52,NaN,3.61,2.48,1.8713,1760.2500,198.3500,105.340,304.4200,324.605
2019/12/16 11:00,15.90,2.37,1.19,4.10,2.70,1.8748,1763.5000,198.1668,105.260,305.3000,324.670
2019/12/16 11:10,15.70,2.13,1.23,4.00,2.30,1.8818,1762.2100,197.6800,105.360,304.9263,324.140
2019/12/16 11:20,15.35,2.21,1.34,3.60,2.65,1.8818,1761.5074,197.7500,105.537,304.5448,324.410
2019/12/16 11:30,15.11,1.95,1.28,3.81,2.58,1.8801,1760.6401,197.2556,105.400,304.9036,324.435


In [6]:
from scipy.optimize import minimize
from scipy.stats import norm

def black_scholes(s,k,rf,t,sigma):
    d1 = (np.log(s/k)+(rf+0.5*sigma**2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    p = s*norm.cdf(d1)-k*np.exp(-rf*t)*norm.cdf(d2)
    return p

def get_delta(s,k,rf,t,sigma):
    d1 = (np.log(s/k)+(rf+0.5*sigma**2)*t)/(sigma*np.sqrt(t))
    return norm.cdf(d1)
    
def price_diff(args,s,k,rf,p,t):
    sigma = args[0]
    p_bs = black_scholes(s,k,rf,t,sigma)
    return abs(p-p_bs)

def get_implied_volitality(s,k,rf,p,t):
    sigma_0 = 0.2
    res = minimize(price_diff, [sigma_0],(s,k,rf,p,t) , method = "nelder-mead",
                options={"xtol": 1e-8, "disp": False, "maxiter": 1000})
    return res.x[0],res.fun

def time_diff(t):
    t = datetime.strptime(t,"%Y/%m/%d %H:%M")
    tdiff = opt_expire_time - t
    res = tdiff.days + tdiff.seconds/86400
    return res/365

In [7]:
from tqdm.notebook import tqdm

for col in tqdm(opt_data.columns[:5]):
    sname = col.split("_")[0]
    iv_col_name = f"{sname}_IV"
    iv_list = []
    del_list = []
    del_col_name = f"{sname}_del"

    for row in opt_data.index:
        p = opt_data.loc[row, col]
        if np.isnan(p):
            iv_list.append(np.nan)
            del_list.append(np.nan)
            continue
        s = opt_data.loc[row, sname]
        k = strike_price_dict[sname]
        rf = opt_data.loc[row, "rf"] / 100
        t = time_diff(row)
        impvol, fun = get_implied_volitality(s, k, rf, p, t)
        iv_list.append(impvol if fun < 0.01 else np.nan)
        delta = get_delta(s, k, rf, t, impvol) if fun < 0.01 else np.nan
        del_list.append(delta)

    opt_data[iv_col_name] = iv_list
    opt_data[del_col_name] = del_list


  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
opt_data.head()

,AMZN_OPT,FB_OPT,EA_OPT,NFLX_OPT,ADBE_OPT,rf,AMZN,FB,EA,NFLX,...,AMZN_IV,AMZN_del,FB_IV,FB_del,EA_IV,EA_del,NFLX_IV,NFLX_del,ADBE_IV,ADBE_del
DateTime,,,,,,,,,,,,,,,,,,,,,
2019/10/30 9:30,NaN,NaN,0.53,6.56,NaN,1.8191,1762.0000,187.7200,93.630,286.9500,...,NaN,NaN,NaN,NaN,0.250762,0.126045,0.305054,0.324895,NaN,NaN
2019/10/30 9:40,NaN,NaN,0.60,6.96,NaN,1.8103,1768.6140,188.4900,94.210,288.7200,...,NaN,NaN,NaN,NaN,0.249844,0.139159,0.300269,0.341415,NaN,NaN
2019/10/30 9:50,NaN,NaN,0.56,NaN,NaN,1.8138,1766.4446,188.1050,93.140,289.1200,...,NaN,NaN,NaN,NaN,0.263477,0.127403,NaN,NaN,NaN,NaN
2019/10/30 10:00,NaN,NaN,0.46,7.24,NaN,1.8138,1766.4600,187.8081,92.770,287.8700,...,NaN,NaN,NaN,NaN,0.255469,0.111390,0.314678,0.341030,NaN,NaN
2019/10/30 10:10,NaN,NaN,0.45,NaN,NaN,1.8120,1768.1724,187.9650,93.085,288.8869,...,NaN,NaN,NaN,NaN,0.248594,0.111379,NaN,NaN,NaN,NaN


In [9]:
opt_data.columns

Index(['AMZN_OPT', 'FB_OPT', 'EA_OPT', 'NFLX_OPT', 'ADBE_OPT', 'rf', 'AMZN',
       'FB', 'EA', 'NFLX', 'ADBE', 'AMZN_IV', 'AMZN_del', 'FB_IV', 'FB_del',
       'EA_IV', 'EA_del', 'NFLX_IV', 'NFLX_del', 'ADBE_IV', 'ADBE_del'],
      dtype='object')

In [33]:
from arch import arch_model

# Ensure time sorting
opt_data = opt_data.sort_index()

# Rolling window for benchmark volatility
rolling_window = 100

# GARCH/EGARCH model to apply per stock
best_model = {
    'AMZN': 'GARCH',
    'FB': 'GARCH',
    'EA': 'EGARCH',
    'NFLX': 'EGARCH',
    'ADBE': 'EGARCH'
}

# Model configurations
model_configs = {
    'GARCH': {'vol': 'Garch'},
    'EGARCH': {'vol': 'EGARCH'}
}

# Loop through each ticker
for ticker, model_type in best_model.items():
    print(f"\nProcessing {ticker} with {model_type} model...")
    
    try:
        # Get price and returns
        prices = opt_data[ticker].dropna()
        returns = np.log(prices).diff().dropna() * 100

        # Rolling benchmark volatility
        roll_vol = returns.rolling(window=rolling_window).std()
        opt_data[f"{ticker}_ROLLV"] = roll_vol.reindex(opt_data.index)

        # Fit selected model
        p, q = 5, 5
        model = arch_model(returns, p=p, q=q, **model_configs[model_type], rescale=False)
        res = model.fit(disp='off')
        cond_vol = res.conditional_volatility

        # Store predicted volatility in GARCHV column
        full_vol = pd.Series(np.nan, index=opt_data.index)
        full_vol.loc[cond_vol.index] = cond_vol.values
        opt_data[f"{ticker}_GARCHV"] = full_vol

    except Exception as e:
        print(f"⚠️ Error for {ticker}: {e}")



Processing AMZN with GARCH model...

Processing FB with GARCH model...

Processing EA with EGARCH model...

Processing NFLX with EGARCH model...

Processing ADBE with EGARCH model...


In [11]:
opt_data.tail(10)

,AMZN_OPT,FB_OPT,EA_OPT,NFLX_OPT,ADBE_OPT,rf,AMZN,FB,EA,NFLX,...,AMZN_ROLLV,AMZN_GARCHV,FB_ROLLV,FB_GARCHV,EA_ROLLV,EA_GARCHV,NFLX_ROLLV,NFLX_GARCHV,ADBE_ROLLV,ADBE_GARCHV
DateTime,,,,,,,,,,,,,,,,,,,,,
2019/12/9 14:50,20.70,NaN,NaN,5.64,NaN,1.8294,1748.7800,202.2932,102.06,303.5100,...,0.117934,0.175225,0.158537,0.250425,0.324951,0.253359,0.239299,0.522115,0.198915,0.221397
2019/12/9 15:00,20.80,NaN,NaN,5.75,1.21,1.8277,1749.7567,202.2500,102.13,303.9500,...,0.118059,0.174279,0.157983,0.230054,0.324968,0.268612,0.239637,0.533842,0.197433,0.204467
2019/12/9 15:10,21.00,NaN,NaN,5.71,NaN,1.8294,1750.9600,202.1800,102.12,303.6700,...,0.118228,0.176190,0.157783,0.262905,0.324934,0.321244,0.237930,0.516766,0.197335,0.310622
2019/12/9 15:20,NaN,NaN,NaN,5.65,NaN,1.8312,1750.5031,201.9385,102.07,303.4200,...,0.117763,0.176895,0.158340,0.233238,0.324830,0.322349,0.238087,0.556829,0.197330,0.217088
2019/12/9 15:30,21.25,NaN,NaN,5.50,1.13,1.8312,1750.5400,201.5600,101.95,303.1000,...,0.117631,0.178193,0.159193,0.218130,0.325039,0.251349,0.236348,0.554888,0.198455,0.164419
2019/12/9 15:40,20.20,5.93,0.57,5.35,1.10,1.8312,1749.0000,201.3700,101.99,303.2000,...,0.117976,0.180204,0.159523,0.234466,0.324995,0.251718,0.231602,0.542895,0.198439,0.229954
2019/12/9 15:50,20.30,5.60,NaN,5.25,1.08,NaN,1749.5100,201.3400,102.03,302.5000,...,0.118005,0.175137,0.159274,0.265273,0.325002,0.265222,0.232582,0.545228,0.198466,0.265066
2019/12/9 9:30,27.35,5.75,0.79,9.28,1.37,1.8173,1761.0919,201.5150,102.56,309.5485,...,0.135106,0.174188,0.159036,0.240956,0.328907,0.331089,0.326973,0.573678,0.215750,0.925858
2019/12/9 9:40,29.55,6.22,0.78,8.81,1.51,1.8208,1764.5551,202.0700,103.13,308.1300,...,0.136205,0.190434,0.160821,0.220922,0.333322,0.306463,0.330480,0.770445,0.215580,0.199523


In [12]:
# import matplotlib.pyplot as plt

# plt.subplots(3, 2, figsize=(20, 15))

# # Extract tickers from column names (e.g., 'EA_IV' → 'EA')
# iv_cols = [col for col in opt_data.columns if col.endswith('_IV')]
# tickers = [col.split('_')[0] for col in iv_cols]

# for i, ticker in enumerate(tickers):
#     iv_col = f"{ticker}_IV"
#     garch_col = f"{ticker}_GARCHV"
#     bench_col = f"{ticker}_ROLLV"

#     plt.subplot(3, 2, i + 1)

#     # Plot Implied Volatility
#     opt_data[iv_col].plot(label="Implied Vol")

#     # Plot GARCH Volatility if available
#     if garch_col in opt_data.columns:
#         opt_data[garch_col].plot(label="GARCH Vol")

#     # Plot Benchmark Rolling Volatility if available
#     if bench_col in opt_data.columns:
#         opt_data[bench_col].plot(label="Benchmark Vol")

#     plt.grid(True)
#     plt.legend()
#     plt.title(f"Volatility: {ticker}  |  IV: {opt_data[iv_col].count()} | GARCH: {opt_data[garch_col].count() if garch_col in opt_data else 0} | BENCH: {opt_data[bench_col].count() if bench_col in opt_data else 0}")

# plt.tight_layout()
# plt.show()


In [24]:
import numpy as np

# Updated backtest function
def backtest(stk, model='GARCHV'):
    pred_vol_col = f"{stk}_{model}"
    iv_col = f"{stk}_IV"
    opt_col = f"{stk}_OPT"
    spot_col = stk
    delta_col = f"{stk}_del"

    # Drop missing values
    tmp = opt_data[[iv_col, pred_vol_col, opt_col, spot_col, delta_col]].dropna()
    tmp.columns = ["iv", "predicted_vol", "opt_price", "spot", "delta"]

    # Difference between implied and predicted vol
    tmp["diff"] = tmp["iv"] - tmp["predicted_vol"]

    # Strategy logic
    action = []
    snumber = []
    state = 0
    for i, row in tmp.iterrows():
        d = row["diff"]
        delta = row["delta"]
        if d > 0 and state == 0:
            action.append(-1)
            snumber.append(delta)
            state = -1
        elif d < 0 and state == 0:
            action.append(1)
            snumber.append(-delta)
            state = 1
        elif d < 0 and state == -1:
            action.append(1)
            snumber.append(snumber[-1])
            state = 0
        elif d > 0 and state == 1:
            action.append(-1)
            snumber.append(snumber[-1])
            state = 0
        else:
            action.append(0)
            snumber.append(snumber[-1] if snumber else 0)

    tmp["action"] = action
    tmp["snumber"] = snumber
    tmp = tmp[tmp["action"] != 0]

    if len(tmp) % 2 == 1:
        tmp = tmp.iloc[:-1]

    res_h = []
    timestamps = []
    for i in range(1, len(tmp), 2):
        opt_pnl = -tmp["opt_price"].iloc[i] * tmp["action"].iloc[i] - tmp["opt_price"].iloc[i - 1] * tmp["action"].iloc[i - 1]
        spot_pnl = (tmp["spot"].iloc[i] - tmp["spot"].iloc[i - 1]) * tmp["snumber"].iloc[i]
        res_h.append(opt_pnl + spot_pnl)
        timestamps.append(tmp.index[i])  # Use close time of trade pair

    return list(zip(timestamps, res_h))

In [27]:
def show_backtest_result(stock_list, model='GARCHV'):
    all_returns = []
    all_rf = []

    print(f"\n{model}")
    print("=== Individual Results ===")

    for stk in stock_list:
        results = backtest(stk, model)
        if results:
            times, res = zip(*results)
            res = np.array(res)

            # Ensure rf is aligned and divided by 100 if it's in %
            rf_values = opt_data.loc[list(times), 'rf'].values / 100
            excess_returns = res - rf_values

            # Downside deviation for Sortino Ratio
            downside = excess_returns[excess_returns < 0]
            sortino_denom = np.std(downside) if len(downside) > 0 else 0

            total_ret = round(np.sum(res), 3)
            avg_ret = round(np.mean(res), 3)
            std_ret = round(np.std(excess_returns), 3)
            sharpe = round(np.mean(excess_returns) / std_ret, 3) if std_ret != 0 else 'NA'
            sortino = round(np.mean(excess_returns) / sortino_denom, 3) if sortino_denom != 0 else 'NA'
            win_rate = round(np.sum(res > 0) / len(res), 3)
            profit_factor = round(np.sum(res[res > 0]) / abs(np.sum(res[res < 0])), 3) if np.any(res < 0) else 'Inf'

            print(f"{stk}: Trades = {len(res)}, Total Return = {total_ret}, Avg Return = {avg_ret}, "
                  f"Sharpe = {sharpe}, Sortino = {sortino}, Win Rate = {win_rate}, Profit Factor = {profit_factor}")

            all_returns.extend(res)
            all_rf.extend(rf_values)
        else:
            print(f"{stk}: No trades executed.")

    print("\n=== Combined Strategy Results ===")
    if all_returns:
        all_returns = np.array(all_returns)
        all_rf = np.array(all_rf)
        excess = all_returns - all_rf

        downside = excess[excess < 0]
        sortino_denom = np.std(downside) if len(downside) > 0 else 0

        total_ret = round(np.sum(all_returns), 3)
        avg_ret = round(np.mean(all_returns), 3)
        std_ret = round(np.std(excess), 3)
        sharpe = round(np.mean(excess) / std_ret, 3) if std_ret != 0 else 'NA'
        sortino = round(np.mean(excess) / sortino_denom, 3) if sortino_denom != 0 else 'NA'
        win_rate = round(np.sum(all_returns > 0) / len(all_returns), 3)
        profit_factor = round(np.sum(all_returns[all_returns > 0]) / abs(np.sum(all_returns[all_returns < 0])), 3) if np.any(all_returns < 0) else 'Inf'

        print(f"Cumulative Return of the strategy: {total_ret}")
        print(f"Average Return per Trade: {avg_ret}")
        print(f"Standard Deviation of Excess Returns: {std_ret}")
        print(f"Sharpe Ratio (Excess): {sharpe}")
        print(f"Sortino Ratio (Excess): {sortino}")
        print(f"Win Rate: {win_rate}")
        print(f"Profit Factor: {profit_factor}")
        print(f"Total Number of Trades: {len(all_returns)}")
    else:
        print("No trades executed for any stock.")


In [34]:
all_tickers = ['AMZN', 'FB', 'EA', 'NFLX', 'ADBE']
show_backtest_result(all_tickers, 'GARCHV')
show_backtest_result(all_tickers, 'ROLLV')


GARCHV
=== Individual Results ===
AMZN: Trades = 52, Total Return = 44.388, Avg Return = 0.854, Sharpe = 0.52, Sortino = 1.729, Win Rate = 0.692, Profit Factor = 5.068
FB: Trades = 101, Total Return = 7.451, Avg Return = 0.074, Sharpe = 0.156, Sortino = 0.459, Win Rate = 0.545, Profit Factor = 2.368
EA: Trades = 27, Total Return = 1.747, Avg Return = 0.065, Sharpe = 0.102, Sortino = 0.202, Win Rate = 0.407, Profit Factor = 1.938
NFLX: Trades = 77, Total Return = 4.694, Avg Return = 0.061, Sharpe = 0.038, Sortino = 0.053, Win Rate = 0.455, Profit Factor = 1.353
ADBE: Trades = 78, Total Return = 11.767, Avg Return = 0.151, Sharpe = 0.109, Sortino = 0.809, Win Rate = 0.487, Profit Factor = 3.153

=== Combined Strategy Results ===
Cumulative Return of the strategy: 70.047
Average Return per Trade: 0.209
Standard Deviation of Excess Returns: 1.085
Sharpe Ratio (Excess): 0.176
Sortino Ratio (Excess): 0.396
Win Rate: 0.522
Profit Factor: 2.894
Total Number of Trades: 335

ROLLV
=== Individua